In [2]:
from keras.models import load_model
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from keras.models import save_model
import matplotlib.pyplot as plt


In [3]:
data_folder = "images"

class_labels = ["Aadhaar", "PAN", "Driving Licence",
                "Voter ID", "Passport"]


In [4]:
def preprocess_images(folder_path):
    images = []
    labels = []
    for i, label in enumerate(class_labels):
        folder = os.path.join(folder_path, label)
        for filename in os.listdir(folder):
            img_path = os.path.join(folder, filename)
            img = cv2.imread(img_path)
            if img is not None and img.size != 0:  # Check for valid image size
                img = cv2.resize(img, (224, 224))
                img = img / 255.0
                images.append(img)
                labels.append(i)

    return np.array(images), np.array(labels)

  
images, labels = preprocess_images(data_folder)


In [5]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    images, labels, test_size=0.2, random_state=42)

# Build the CNN model
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(len(class_labels), activation='softmax'))


In [6]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 222, 222, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 111, 111, 32)      0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 109, 109, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 54, 54, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 52, 52, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 26, 26, 128)       0

In [7]:
history= model.fit(X_train, y_train, epochs=8, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/8
13/13 [==============================] - 37s 2s/step - loss: 2.7555 - accuracy: 0.2134 - val_loss: 1.6166 - val_accuracy: 0.1782
Epoch 2/8
13/13 [==============================] - 30s 2s/step - loss: 1.4984 - accuracy: 0.3325 - val_loss: 1.4391 - val_accuracy: 0.4554
Epoch 3/8
13/13 [==============================] - 29s 2s/step - loss: 0.9585 - accuracy: 0.6476 - val_loss: 1.0647 - val_accuracy: 0.6436
Epoch 4/8
13/13 [==============================] - 36s 3s/step - loss: 0.5068 - accuracy: 0.8412 - val_loss: 0.7073 - val_accuracy: 0.7624
Epoch 5/8
13/13 [==============================] - 31s 2s/step - loss: 0.3794 - accuracy: 0.8883 - val_loss: 0.7390 - val_accuracy: 0.8119
Epoch 6/8
13/13 [==============================] - 26s 2s/step - loss: 0.2101 - accuracy: 0.9305 - val_loss: 0.7665 - val_accuracy: 0.7822
Epoch 7/8
13/13 [==============================] - 26s 2s/step - loss: 0.1080 - accuracy: 0.9677 - val_loss: 0.6791 - val_accuracy: 0.8218
Epoch 8/8
13/13 [==========

In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 222, 222, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 111, 111, 32)      0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 109, 109, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 54, 54, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 52, 52, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 26, 26, 128)       0

In [8]:
# Evaluate the model on the testing set
loss, accuracy = model.evaluate(X_test, y_test)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)




4/4 [==============================] - 2s 353ms/step - loss: 0.7236 - accuracy: 0.8020
Test Loss: 0.7235658764839172
Test Accuracy: 0.801980197429657


d:\Users\r111\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
# Save the trained model
model.save("classifier_model.h5")
